In [92]:
import numpy as np 
import pandas as pd 
import random
from collections import Counter

In [18]:
class colors():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['♠', '♦', '♥', '♣'] #['S','H','D','C'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [3]:
class numbers():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [12]:
class card():
    __slots__ = '__color', '__number'
    
    def __init__(self,aColor, aNumber):
        
        self.__color = aColor
        self.__number = aNumber
        
    def getColor(self):
        return self.__color
    
    def getNumber(self):
        return self.__number
    

In [41]:
class hand():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            

        

In [50]:
class deck():
    __slots__ = '__cards'
        
    def __init__(self):
        self.__cards = []
        for color in colors().getPossibilities():
            for number in numbers().getPossibilities():
                self.__cards.append(card(color, number))
        random.shuffle(self.__cards)
    
    def drawCard(self):
        return self.__cards.pop(0)
    
    def getDeck(self):
        return self.__cards

In [43]:
class river():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            


In [142]:
def isFlush(aHand):
    
    myMaxColor = max(Counter(myBigHands[0].getColors()))
      
    return myBigHands[0].getColors()[myMaxColor] >= 5, myMaxColor == '♥' #check for royal

def isStraight(aHand):
    first = True
    countConseq = 0
    maxConSeq = 0
    lastCardSeq = None
    myIndexedSortedHand = sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])
    for idx in myIndexedSortedHand:
        if not first:
            if idx == myLastIdx+1:
                countConseq = countConseq+1
                lastCardSeq = myLastIdx #to compare between straights
            else:
                countConseq = 0
            if countConseq > maxConSeq :
                maxConSeq = countConseq
        first = False
        myLastIdx = idx
    
    return maxConSeq>=5,lastCardSeq
    
    
def howManySameNumber(aHand):
    Counter(aHand.getNumbers())
    
def singleCard(aHand, idxBest): #idxBest starts at 1 for best, 2 for snd Best
    
    return sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-idx] #get number with highest

In [144]:
Counter(aHand.getNumbers()).values()

dict_values([2, 1, 2, 1, 1])

In [ ]:
def getWinner(river, hands):
    #return index of winning hands
    values = []
    
    for hand in hands:
        myHandToEval = [hand.getCards(), river.getCards()]
        values.append(getValueHand(hand))
        
        
    
    

In [9]:
a = ['K', 'J', 'A','A'] 
possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A']

sorted(np.unique(a), key=lambda x: possibilities.index(x))

['J', 'K', 'A']

In [84]:
myDeck = deck()
mySmallHands = [] #individualHand
myBigHands = [] #Hand with River
for i in range(0,2):
    mySmallHands.append(hand())
    myBigHands.append(hand())
myRiver = river()

In [85]:
#draw cards
for drawnCard in range(0,5):
    for player in range(0,2):
        myCard = myDeck.drawCard()
        mySmallHands[player].addCard(myCard)
        myBigHands[player].addCard(myCard)
        
#draw cards in the river
for drawnCard in range(0,2):
    myCard = myDeck.drawCard()
    
    myRiver.addCard(myCard)
    for player in range(0,2):
        myBigHands[player].addCard(myCard)

In [115]:
aHand = myBigHands[1]
sorted(np.unique(aHand.getNumbers()), key=lambda x: numbers().getPossibilities().index(x))[-1]

'A'

In [129]:
Counter(aHand.getNumbers())

Counter({'2': 1, '3': 2, '6': 1, 'A': 1, 'K': 2})

In [138]:
sorted(np.unique(aHand.getNumbers()), key=lambda x: numbers().getPossibilities().index(x))

[0, 1, 1, 4, 10, 10, 11]

In [136]:
np.unique(aHand.getNumbers())

array(['2', '3', '6', 'A', 'K'], 
      dtype='<U1')

In [134]:
[numbers().getPossibilities().index(x) for x in aHand.getNumbers()]

[1, 11, 1, 10, 10, 0, 4]

In [141]:
sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-1]

11